In [2]:
import numpy as np
import pandas as pd
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
import random as rand

In [6]:
df = pd.read_csv("8G1.csv")
G0 = nx.read_gexf("graph/G0.gexf")
G1 = nx.read_gexf("graph/G1.gexf")

In [4]:
df.head(5)

,date,adr,p,n_killed,n_injured,city,state,LON,LAT,log_n_killed,log_n_injured,indegree,outdegree
ind_t,,,,,,,,,,,,,
0,2014-01-01,Rockingham Street and Berkley Avenue Extended ...,1.098612,2,2,Norfolk,Virginia,-76.278935,36.830788,1.098612,1.098612,0,8
1,2014-01-03,Farmers Boulevard and 133rd Avenue Queens New ...,0.901091,1,3,Queens,New York,-73.762898,40.677500,0.693147,1.386294,1,6
2,2014-01-05,829 Parade St Erie Pennsylvania,0.901091,1,3,Erie,Pennsylvania,-80.074524,42.130038,0.693147,1.386294,1,7
3,2014-01-11,3430 W. Capitol Street Jackson Mississippi,0.482831,0,4,Jackson,Mississippi,-90.228321,32.323689,0.000000,1.609438,3,0
4,2014-01-12,University Drive Huntsville Alabama,0.537528,0,5,Huntsville,Alabama,-86.650390,34.736562,0.000000,1.791759,3,4


### shuffle time index, (ind_t)

In [7]:
### create new time index
ind_t_list = list(df['ind_t'])

numbers = []
for i in range(0, 1206):
    numbers.append(i)

rand.seed(99)
num = rand.sample(numbers, 1206)

new_idx = []

for i in range(len(ind_t_list)):
    for j in range(len(num)):
        if ind_t_list[i] == j:
            new_t = num[j]
            new_idx.append(new_t)
            
df['rand_ind_t'] = new_idx
df.to_csv("9rand_t_index.csv", index = False)

### create null_model

In [10]:
rand_df = pd.read_csv("9rand_t_index.csv")
sorted_rand_df = rand_df.sort_values(by = 'rand_ind_t', ascending = True)

In [11]:
sorted_rand_df.head(5)

,ind_t,date,adr,p,n_killed,n_injured,city,state,LON,LAT,log_n_killed,log_n_injured,indegree,outdegree,rand_ind_t
962,564,2016-12-05,1532 Stagecoach Lane Albuquerque New Mexico,1.334551,4,1,Albuquerque,New Mexico,-106.490998,35.048201,1.609438,0.693147,0,13,0
1526,890,2018-07-28,8801 S Vermont Ave Los Angeles California,1.251860,2,4,Los Angeles,California,-118.292011,33.957716,1.098612,1.609438,2,16,1
1527,890,2018-07-28,3400 S Claiborne Ave New Orleans Louisiana,1.594239,3,7,New Orleans,Louisiana,-90.097796,29.940498,1.386294,2.079442,0,17,1
265,170,2014-12-26,2500 block of Summit Avenue East St. Louis Ill...,0.901091,1,3,East St. Louis,Illinois,-90.125416,38.619618,0.693147,1.386294,5,3,2
741,450,2016-06-13,Avenue D and E. 57th Brooklyn New York,0.537528,0,5,Brooklyn,New York,-73.922619,40.642746,0.000000,1.791759,8,6,3


### Null

In [13]:
G1_null = nx.DiGraph()

for n in range(0, len(df), 1):
  G1_null.add_node(n,
             rand_ind_t = sorted_rand_df['rand_ind_t'][n],
             p = sorted_rand_df['p'][n],
             n_killed = sorted_rand_df['log_n_killed'][n],
             n_injured = sorted_rand_df['log_n_injured'][n],
             LON = sorted_rand_df['LON'][n],
             LAT = sorted_rand_df['LAT'][n])

for i in G1_null.nodes():
  for j in G1_null.nodes():
    if 0 < G1_null.nodes[j]['rand_ind_t']-G1_null.nodes[i]['rand_ind_t'] < 7 and G1_null.nodes[i]['p'] > G1_null.nodes[j]['p']:
      G1_null.add_edge(i,j)
    else:
      pass

In [14]:
nx.info(G1)

'Name: \nType: DiGraph\nNumber of nodes: 2087\nNumber of edges: 9453\nAverage in degree:   4.5295\nAverage out degree:   4.5295'

In [15]:
nx.info(G1_null)

'Name: \nType: DiGraph\nNumber of nodes: 2087\nNumber of edges: 9449\nAverage in degree:   4.5276\nAverage out degree:   4.5276'

In [16]:
nx.write_gexf(G1_null, "graph/G1_null.gexf")

In [28]:
G1_null_inDgr = []

for i in list(G1_null.nodes()):
    G1_null_inDegree = G1_null.in_degree(i)
    G1_null_inDgr.append(G1_null_inDegree)
    
sorted_rand_df['null_indegree'] = G1_null_inDgr

G1_null_outDgr = []

for i in list(G1_null.nodes()):
    G1_null_outDegree = G1_null.out_degree(i)
    G1_null_outDgr.append(G1_null_outDegree)
    
sorted_rand_df['null_outdegree'] = G1_null_outDgr

In [23]:
sorted_rand_df = df.sort_values(by = 'rand_ind_t', ascending = True)

In [29]:
sorted_rand_df.head(5)

,ind_t,date,adr,p,n_killed,n_injured,city,state,LON,LAT,log_n_killed,log_n_injured,indegree,outdegree,rand_ind_t,null_indegree,null_outdegree
962,564,2016-12-05,1532 Stagecoach Lane Albuquerque New Mexico,1.334551,4,1,Albuquerque,New Mexico,-106.490998,35.048201,1.609438,0.693147,0,13,0,2,5
1526,890,2018-07-28,8801 S Vermont Ave Los Angeles California,1.251860,2,4,Los Angeles,California,-118.292011,33.957716,1.098612,1.609438,2,16,1,3,3
1527,890,2018-07-28,3400 S Claiborne Ave New Orleans Louisiana,1.594239,3,7,New Orleans,Louisiana,-90.097796,29.940498,1.386294,2.079442,0,17,1,2,4
265,170,2014-12-26,2500 block of Summit Avenue East St. Louis Ill...,0.901091,1,3,East St. Louis,Illinois,-90.125416,38.619618,0.693147,1.386294,5,3,2,9,0
741,450,2016-06-13,Avenue D and E. 57th Brooklyn New York,0.537528,0,5,Brooklyn,New York,-73.922619,40.642746,0.000000,1.791759,8,6,3,6,3


In [36]:
print(sorted_rand_df['log_n_killed'].corr(sorted_rand_df['indegree']), 
     sorted_rand_df['log_n_killed'].corr(sorted_rand_df['null_indegree']))

-0.7384716125256426 0.006615872549785111


In [37]:
print(sorted_rand_df['log_n_injured'].corr(sorted_rand_df['indegree']), 
     sorted_rand_df['log_n_injured'].corr(sorted_rand_df['null_indegree']))

0.2936088237473305 -0.027869897123153162


In [38]:
print(sorted_rand_df['log_n_killed'].corr(sorted_rand_df['outdegree']), 
     sorted_rand_df['log_n_killed'].corr(sorted_rand_df['null_outdegree']))

0.742305314898483 -0.032660284849324775


In [39]:
print(sorted_rand_df['log_n_injured'].corr(sorted_rand_df['outdegree']), 
     sorted_rand_df['log_n_injured'].corr(sorted_rand_df['null_outdegree']))

-0.25859344467657003 0.03434304458551898


In [41]:
sorted_rand_df.to_csv("9with_null_model.csv", index = False)